In [ ]:
import torch
from pathlib import Path
from PIL import Image, ImageFilter
import numpy as np


In [ ]:
DATA_DIR = Path("../data/images")
png_paths = sorted(DATA_DIR.glob("**/*.png"))

if not png_paths:
    print(f"No PNG files found in {DATA_DIR.resolve()}")
else:
    pairs = {}
    for path in png_paths:
        stem = path.stem
        try:
            base, eye = stem.rsplit("_", 1)
        except ValueError:
            print(f"Skipping {path.name}: unable to parse eye suffix")
            continue
        eye = eye.lower()
        if eye not in {"left", "right"}:
            print(f"Skipping {path.name}: unexpected eye tag {eye!r}")
            continue
        pairs.setdefault(base, {})[eye] = path

    stacks = []
    metadata = []
    for base, eyes in sorted(pairs.items()):
        if {"left", "right"} <= eyes.keys():
            with Image.open(eyes["left"]) as left_img:
                left_gray = left_img.convert("L")
                left_blurred = left_gray.filter(ImageFilter.GaussianBlur(radius=1.5))
                left_array = np.array(left_blurred, dtype=np.float32) / 255.0

            with Image.open(eyes["right"]) as right_img:
                right_gray = right_img.convert("L")
                right_blurred = right_gray.filter(ImageFilter.GaussianBlur(radius=1.5))
                right_mirrored = right_blurred.transpose(Image.FLIP_LEFT_RIGHT)
                right_array = np.array(right_mirrored, dtype=np.float32) / 255.0

            left_tensor = torch.from_numpy(left_array).unsqueeze(0)
            right_tensor = torch.from_numpy(right_array).unsqueeze(0)
            stacked = torch.stack([left_tensor, right_tensor], dim=0)
            stacks.append(stacked)
            metadata.append(base)
        else:
            missing = {"left", "right"} - eyes.keys()
            missing_list = ", ".join(sorted(missing))
            print(f"Skipping {base}: missing {missing_list} eye image")

    if stacks:
        image_stack = torch.stack(stacks)
        print("Image stack shape:", tuple(image_stack.shape))
        print("First few entries:", metadata[:5])
    else:
        print("No complete left/right eye pairs found.")
